In [ ]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
sys.path.append("codes/modules")
#########################################

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
from load_mesh_data_new import *
import numpy as np
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
import colormaps as cmaps
from matplotlib.colors import ListedColormap

In [ ]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
sys.path.append("codes/modules")
#########################################

In [ ]:
# PLOT CONFIGURATION #

# CSV export
export_csv = True
# figure export definition
dpicnt=150
# choose simulation
runid='Arc12'
# choose date
first_year = 1990
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)
# choose depth
get3d = False
# load the given physical variables #
var_id1, var_id2 = 'u','v'

########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'
outputpath = '/scratch/usr/hbkoziel/Arc12/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

In [ ]:
mesh = fesom_mesh(meshpath, get3d = get3d)

In [ ]:
# loop over years
for mo in range(4,10):
    # initlialyze
    U = np.zeros((len(years),len(mesh.x2)))
    V = np.zeros((len(years),len(mesh.x2)))
    EKE = np.zeros((len(years),len(mesh.x2)))
    if mo ==4: 
     dayind = (range(45,59))
     month='APR'
    elif mo ==5: 
     dayind = (range(60,74))
     month='MAY'
    elif mo ==6: 
     dayind = (range(75,89))
     month='JUN'
    elif mo ==7: 
     dayind = (range(90,105))
     month='JUL'
    elif mo ==8:
     dayind = (range(106,120))
     month='AUG'
    elif mo ==9: 
     dayind = (range(121,135))
     month='SEP'
    print month
    for ind in range(0,len(years)):
        print years[ind]
        ncfile = resultpath+runid+'.'+str(years[ind])+'.oce.mean.nc'
        f      = Dataset(ncfile, 'r')
        if years[ind]<2000:
            U[ind,:]    = f.variables[var_id1][mo,0:mesh.n2d]
            V[ind,:]    = f.variables[var_id2][mo,0:mesh.n2d]
        else:
            U[ind,:]    = f.variables[var_id1][dayind,0:mesh.n2d].mean(axis=0)
            V[ind,:]    = f.variables[var_id2][dayind,0:mesh.n2d].mean(axis=0)  

    for ind in range(0,len(years)):
        EKE[ind,:] = 0.5 * (U.mean(axis=0)**2+V.mean(axis=0)**2) - 0.5 * (U[ind,:]**2 + V[ind,:]**2)
        data2 = EKE.mean(axis=0)
    
    if export_csv == True:
        np.savetxt(outputpath+'EKEsurf_'+month+'_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
        np.savetxt(outputpath+'EKEsurf_'+month+'_'+str(first_year)+'_'+str(last_year)+'_trend.csv', EKE, delimiter=";")
        print 'exporting CSV done...'